In [1]:
# Import system packages

import os
import operator
import re

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusid import ApiException
from lusidtools.cocoon.utilities import create_scope_id
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import format_instruments_response, format_portfolios_response, format_transactions_response
import pandas as pd
import json
import openpyxl
import pytz
from datetime import datetime,timezone
from lusidtools.cocoon.seed_sample_data import seed_data

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

In [2]:
# Create a new scope

scope = "booking_sample"
portfolio_code = "EQUITY_UK"

# Create a new portfolio

In [17]:
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Create portfolio with properties
created_date = "2019-01-02T00:00:00.0000000+00:00"
# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="LUSID's top 10 FTSE stock portfolio",
    code=portfolio_code,
    base_currency="GBP",
    created=created_date
)

# Upload new portfolio to LUSID
results = []
try:
    response = transaction_portfolios_api.create_portfolio(
        scope=scope, create_transaction_portfolio_request=portfolio_request
    )
    results.append({"success": True, "Message": f"portfolio '{response.id.code}', in scope {scope} created"})

except ApiException as e:
    if(e.body):
        if json.loads(e.body)["name"] != "PortfolioWithIdAlreadyExists":
            raise
        results.append({"success": False, "error": json.loads(e.body)['name'], "Error Message": json.loads(e.body)['title']})
        

pd.DataFrame(data=results)

,success,Message
0,True,"portfolio 'EQUITY_UK', in scope booking_sample..."


# Define Transaction property data types

In [11]:
data_types_api = api_factory.build(lusid.api.DataTypesApi)

data_type_requests = [
                models.CreateDataTypeRequest(
                    scope=scope,
                    code='SettlementStatusCodes',
                    type_value_range="Open",
                    display_name="Available SettlementStatusCodes",
                    description= 'This data type is a list of possible values used for the Settlement Status property',
                    value_type= "String",
                    acceptable_values= [
                        "MessageSentToCustodian"
                        "Settled",
                        "Unsettled",
                        "FailedSettlement",
                        "AckRecFromCustodian"
                    ],
                    unit_schema="NoUnits"                    
                ),
                models.CreateDataTypeRequest(
                    scope=scope,
                    code='ConfirmationStatusCodes',
                    type_value_range="Open",
                    display_name="Available ConfirmationStatusCodes",
                    description= 'This data type is a list of possible values used for the Confirmation Status property',
                    value_type= "String",
                    acceptable_values= [
                        "TradeConfirmed",
                        "MessageSentToBroker",
                        "AckRecFromBroker",
                        "TradeConfirmed"
                    ],
                    unit_schema="NoUnits"                    
                )

] 

results = []
for data_type in data_type_requests:
    try: 
        data_types_api.create_data_type(create_data_type_request=data_type)
        results.append({"DataType": data_type.code, "success": True})
    except ApiException as e:
        results.append({"DataType": data_type.code,"success": False, "error": json.loads(e.body)['name'], "Error Message": json.loads(e.body)['title']})
        if json.loads(e.body)["name"] != "DuplicateIdFailure":
            raise

pd.DataFrame(data=results)


,DataType,success,error,Error Message
0,SettlementStatusCodes,False,DuplicateIdFailure,"Unable to set alternate ID for this entity, as..."
1,ConfirmationStatusCodes,True,NaN,NaN


# Load instruments

In [8]:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

request = models.InstrumentDefinition(
            name="Aviva",
            identifiers={
                "Isin": models.InstrumentIdValue(
                    value="GB0002162385"),
                "ClientInternal": models.InstrumentIdValue(
                    value="EQ_1234")},
            properties=[
                models.ModelProperty(
                    key=f"Instrument/{scope}/instrument_type",
                    value=models.PropertyValue(label_value="instrument_type"),
                )
            ],
        )

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body={"request_id_1": request}
)

print("Instrument Upserted" if len(instrument_response.failed) < 1 else print("Request failed"))

Instrument Upserted


# Define properties for transactions and instruments

In [12]:
properties_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

properties = [
    models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="ConfirmationStatus",
            value_required=False,
            display_name="ConfirmationStatus",
            data_type_id=models.ResourceId(scope=scope, code="ConfirmationStatusCodes"),
    ),
    models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="SettlementStatus",
            value_required=False,
            display_name="SettlementStatus",
            data_type_id=models.ResourceId(scope=scope, code="SettlementStatusCodes"),
    ),
    models.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="instrument_type",
            value_required=False,
            display_name="instrument_type",
            data_type_id=models.ResourceId(scope="system", code="string"),
    )
]

results = []

for prop in properties:
    try:
        # Call LUSID to create a property
        properties_api.create_property_definition(
            prop
        )
        results.append({"success": True, "Message": "Property created"})
    except ApiException as e:
        results.append({"success": False, "error": json.loads(e.body)['name'], "Error Message": json.loads(e.body)['title']})
        if json.loads(e.body)["name"] != "PropertyAlreadyExists":
            raise

pd.DataFrame(data=results)

,success,Message,error,Error Message
0,True,Property created,NaN,NaN
1,False,NaN,PropertyAlreadyExists,Error creating Property Definition 'Transactio...
2,False,NaN,PropertyAlreadyExists,Error creating Property Definition 'Instrument...


# Load transactions

In [18]:
transaction_request = models.TransactionRequest(
            transaction_id="T1",
            type="TransferIn",
            instrument_identifiers={f"instrument/default/ClientInternal": "EQ_1234"},
            transaction_date= "2020-01-02T00:00:00.000000+00:00",
            settlement_date="2020-01-04T00:00:00.000000+00:00",
            units="100",
            total_consideration=models.CurrencyAndAmount(
                amount="1000000", currency="GBP"
            ),
            properties={
                f"Transaction/{scope}/ConfirmationStatus": models.PerpetualProperty(
                    key=f"Transaction/{scope}/ConfirmationStatus",
                    value=models.PropertyValue(
                        label_value="TradeConfirmed"
                    ),
                ),
                f"Transaction/{scope}/SettlementStatus": models.PerpetualProperty(
                    key=f"Transaction/{scope}/SettlementStatus",
                    value=models.PropertyValue(
                        label_value="Unsettled"
                    ),
                )
            },
        )

# Make Upsert Transactions call to LUSID
response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=[transaction_request]
)
print("Transaction loaded")



Transaction loaded
